# A/B Testing for ShoeFly.com
Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

In [1]:
import pandas as pd

1. Examine the first few rows of ad_clicks.

In [2]:
ad_clicks = pd.read_csv("assets/ad_clicks.csv")
ad_clicks.head(10)

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,18:33,A
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B


2. Your manager wants to know which ad platform is getting you the most views. How many views (i.e., rows of the table) came from each utm_source?

In [4]:
ad_clicks.groupby("utm_source")["user_id"].count().reset_index()

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


3. If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed. Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [16]:
ad_clicks["is_click"] = ad_clicks.apply(lambda row: True if not pd.isna(row["ad_click_timestamp"]) else False, axis = 1)
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


4. We want to know the percent of people who clicked on ads from each utm_source. Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. Save your answer to the variable clicks_by_source.

In [17]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


5. Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id. Save your results to the variable clicks_pivot.

In [20]:
clicks_pivot = clicks_by_source.pivot(columns="is_click", index="utm_source", values="user_id").reset_index()

In [21]:
clicks_pivot

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


6. Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source. Was there a difference in click rates for each source?

In [24]:
clicks_pivot["percent_clicked"] = (clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])) * 100
clicks_pivot

is_click,utm_source,False,True,percent_clicked
0,email,175,80,31.372549
1,facebook,324,180,35.714286
2,google,441,239,35.147059
3,twitter,149,66,30.697674


In [25]:
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


7. The column experimental_group tells us whether the user was shown Ad A or Ad B. Were approximately the same number of people shown both ads?

In [26]:
ad_clicks.groupby("experimental_group")["user_id"].count().reset_index()

,experimental_group,user_id
0,A,827
1,B,827


8. Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [29]:
ad_clicks.groupby(["experimental_group", "is_click"])["user_id"].count().reset_index().query('is_click == True')

,experimental_group,is_click,user_id
1,A,True,310
3,B,True,255


9. The Product Manager for the A/B test thinks that the clicks might have changed by day of the week. Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [30]:
a_clicks = ad_clicks[ad_clicks["experimental_group"] == "A"]
b_clicks = ad_clicks[ad_clicks["experimental_group"] == "B"]

In [49]:
ac = a_clicks.groupby(["day", "is_click"])["user_id"].count().reset_index().query("is_click == True")

In [50]:
bc = b_clicks.groupby(["day", "is_click"])["user_id"].count().reset_index().query("is_click == True")

10. Compare the results for A and B. What happened over the course of the week? Do you recommend that your company use Ad A or Ad B?

In [52]:
ab_clicks = pd.DataFrame({"Day": ac["day"], "A Clicks": ac["user_id"], "B Clicks": bc["user_id"]})
ab_clicks

,Day,A Clicks,B Clicks
1,1 - Monday,43,32
3,2 - Tuesday,43,45
5,3 - Wednesday,38,35
7,4 - Thursday,47,29
9,5 - Friday,51,38
11,6 - Saturday,45,42
13,7 - Sunday,43,34
